In [1]:
import torch

# Confirm that the GPU is detected

assert torch.cuda.is_available()

# Get the GPU device name.

device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla T4, n_gpu: 1


In [2]:
!pip install torch
!pip install transformers
!pip install -U -q PyDrive
!pip install datasets
!pip install seqeval
!pip install ray[tune]
!pip install transformers accelerate
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14

In [3]:
import os
import itertools
import pandas as pd
import numpy as np
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("nlpaueb/legal-bert-base-uncased", num_labels=15) # num_labels = 14 + 1


Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

# Importing Datasets

In [4]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/

import json

# from datasets import load_dataset, concatenate_datasets, Features, Value, ClassLabel, Sequence
# dataset = load_dataset('json', data_files='NER_JUDGEMENT_WikiAnn_Format.json')

import pandas as pd
train_df_judgement = pd.read_json('Data_Augmentation_for_Low_Resource_Indian_Legal_NER/NER_TRAIN_JUDGEMENT_PREPROCESSED.json')
train_df_preamble = pd.read_json('Data_Augmentation_for_Low_Resource_Indian_Legal_NER/NER_TRAIN_PREAMBLE_PREPROCESSED.json')

test_df_judgement = pd.read_json('Data_Augmentation_for_Low_Resource_Indian_Legal_NER/NER_DEV_JUDGEMENT_PREPROCESSED.json')
test_df_preamble = pd.read_json('Data_Augmentation_for_Low_Resource_Indian_Legal_NER/NER_DEV_PREAMBLE_PREPROCESSED.json')


Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


In [5]:
legalBARTSamples = pd.read_csv('Data_Augmentation_for_Low_Resource_Indian_Legal_NER/legal_BART_Samples.csv')
legalBARTSamples['tokens'] = legalBARTSamples['tokens'].map(lambda x: eval(x))
legalBARTSamples['ner_tags'] = legalBARTSamples['ner_tags'].map(lambda x: eval(x))

In [6]:
legalBARTSamples

,tokens,ner_tags
0,"[On, specific, query, by, the, Bench, ,, it, w...","[0, 0, 0, 0, 5, 1, 5, 7, 6, 0, 0, 0, 0, 5, 0, ..."
1,"[According, to, the, Agya, ,, span, class, '',...","[0, 5, 5, 3, 5, 0, 2, 0, 0, 0, 0, 0, 0, 0, 7, ..."
2,"[PW3, Vijay, Mishra, ,, Deputy, Manager, ,, HD...","[0, 7, 7, 5, 2, 0, 5, 0, 2, 5, 5, 5, 0, 0, 0, ..."
3,"[He, was, asked, to, come, and, carry, out, th...","[0, 6, 0, 5, 0, 5, 0, 0, 5, 1, 0, 0, 0, 0, 6, ..."
4,"[The, pillion, rider, ,, Satyanarayana, Murthy...","[5, 0, 0, 5, 3, 3, 6, 0, 0, 5, 5, 0, 5, 5, 0, ..."
...,...,...
8389,"[The, Honble, Sri, Justice, Nooty, Ramamohana,...","[2, 0, 0, 2, 2, 2, 2, 0, 0, 0, 1, 0, 0, 0, 1, ..."
8390,"[Petitioner, :, Indira, Nehru, Gandhi, Respond...","[0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, ..."
8391,"[In, The, Court, Of, Sh, ., R.K, ., Sharma, ,,...","[0, 2, 2, 1, 0, 0, 2, 2, 1, 0, 0, 0, 2, 0, 0, ..."
8392,"[Petitioner, :, Bhagwant, Singh, Vs, ., Respon...","[0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, ..."


In [7]:
print(len(train_df_judgement), len(train_df_preamble))
print(len(test_df_judgement), len(test_df_preamble))
total_data = len(train_df_judgement) + len(train_df_preamble) + len(test_df_judgement) + len(test_df_preamble)

8494 553
839 28


In [8]:
# concatenating the judgement + preamble data
df = pd.concat([train_df_judgement, train_df_preamble])
df.reset_index(inplace=True, drop=True)
# print(df)
total_data_original = len(train_df_judgement) + len(train_df_preamble) + len(test_df_judgement) + len(test_df_preamble)
total_data_generated = len(legalBARTSamples)
# splitting train (70% of the total data)

split_original=int(total_data_original*0.7)
# split_generated = int(total_data_generated*0.3)

split_70_original=split_original /(len(train_df_judgement) + len(train_df_preamble))
train_original = df.sample(frac = split_70_original)
val_df = df.drop(train_original.index)


split_generated=int(total_data_generated*0.25)
split_25_generated=split_generated /(len(legalBARTSamples))
train_generated = legalBARTSamples.sample(frac = split_25_generated)

train_df = pd.concat([train_original, train_generated])

test_df = pd.concat([test_df_judgement, test_df_preamble])

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenization

In [9]:
labels_list = [" OTHERS", " PETITIONER", " COURT", " RESPONDENT", " JUDGE", " OTHER_PERSON", " LAWYER", " DATE", " ORG", " GPE", " STATUTE", " PROVISION", " PRECEDENT", " CASE_NUMBER", " WITNESS"]
label_encoding_dict = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14}
label_list_encoding_dict = {0: " OTHERS", 1: " PETITIONER", 2: " COURT", 3: " RESPONDENT", 4: " JUDGE", 5: " OTHER_PERSON", 6: " LAWYER", 7: " DATE", 8: " ORG", 9: " GPE", 10: " STATUTE", 11: " PROVISION", 12: " PRECEDENT", 13: " CASE_NUMBER", 14: " WITNESS"}


In [10]:
def tokenize_all_labels(rows):
    tokenized_inputs = tokenizer(list(rows["tokens"]), truncation = True, is_split_into_words = True)
    labels, label_all = [], True
    for index, label in enumerate(rows["ner_tags"]):
        # print(i, label)
        prior_idx = None
        word_ids = tokenized_inputs.word_ids(batch_index = index)
        
        label_ids = []
        for current_idx in word_ids:
            if current_idx is None: label_ids.append(-100)
            elif label[current_idx] == '0': label_ids.append(0)
            elif current_idx != prior_idx: label_ids.append(label_encoding_dict[label[current_idx]])
            else: label_ids.append(label_encoding_dict[label[current_idx]] if label_all else -100)
            prior_idx = current_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [11]:
train_dataset_tokenized = train_dataset.map(tokenize_all_labels, batched=True)
val_dataset_tokenized = val_dataset.map(tokenize_all_labels, batched=True)

Map:   0%|          | 0/9037 [00:00<?, ? examples/s]

Map:   0%|          | 0/2108 [00:00<?, ? examples/s]

# Training

In [13]:
import wandb
wandb.login()

wandb: Currently logged in as: astha (685_data_augmentation_legal_ner). Use `wandb login --relogin` to force relogin


True

In [14]:
import warnings
warnings.filterwarnings('ignore')

# hyperparameters
batch_size = 5
epoch = 5
learning_rate = 2.910635913133073e-05

# tokenizing and loading legalBERT model 
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModelForTokenClassification.from_pretrained("nlpaueb/legal-bert-base-uncased", num_labels=len(labels_list))
metric = load_metric("seqeval")

training_arguments = TrainingArguments(
    "eval_indian_legal_ner",
    report_to='wandb',
    evaluation_strategy = "steps",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=1e-5,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    run_name='legalBART'  
)

# training on train set and evaluating on validation set
data_collator = DataCollatorForTokenClassification(tokenizer)

best_accuracy = 0
best_results = 0

def evaluate_metrics(pred_tuple):
    global best_accuracy
    global best_results
    predictions, labels = pred_tuple
    predictions = np.argmax(predictions, axis=2)
    
    actual_predictions = [[labels_list[pred_tuple] for (pred_tuple, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    actual_labels = [[labels_list[l] for (pred_tuple, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    results = metric.compute(predictions=actual_predictions, references=actual_labels)
    if best_accuracy < results["overall_accuracy"]: 
      best_results = results
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}
    
trainer = Trainer(
    model,
    training_arguments,
    train_dataset = train_dataset_tokenized,
    eval_dataset = val_dataset_tokenized,
    data_collator = data_collator,
    tokenizer=tokenizer,
    compute_metrics=evaluate_metrics
)
trainer.train()
trainer.evaluate()
trainer.save_model('indian_legal_ner.model')
wandb.finish()

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initia

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.811500,0.386591,0.309715,0.557939,0.398321,0.853700
1000,0.439300,0.295556,0.461010,0.672185,0.546921,0.900688
1500,0.366100,0.259540,0.540371,0.739817,0.624558,0.918099
2000,0.276800,0.207943,0.620310,0.792311,0.695839,0.936724
2500,0.247100,0.174414,0.666036,0.804944,0.728931,0.945369
3000,0.225400,0.216119,0.640049,0.801350,0.711674,0.938729
3500,0.249700,0.237188,0.588736,0.783272,0.672212,0.930732
4000,0.185400,0.224593,0.650152,0.814637,0.723159,0.940368
4500,0.138800,0.252622,0.627901,0.807340,0.706404,0.936664
5000,0.155900,0.206445,0.680398,0.825637,0.746015,0.946705


eval/accuracy,▁▅▆▇█▇▇█▇█▅▇█▇██▇▇█
eval/f1,▁▄▆▇█▇▇█▇█▅▇█▇██▇▇█
eval/loss,▆▄▃▂▁▂▃▂▃▂█▅▃▅▄▄▆▅▂
eval/precision,▁▄▅▇█▇▆▇▇█▄▆█▇▇▇▆▇█
eval/recall,▁▄▆▇▇▇▇███▆▇████▇██
eval/runtime,▂█▆█▂▁▆▆▃▁▁▂▁▁▂▇▇▅█
eval/samples_per_second,▇▁▂▁▆█▂▃▆██▇██▇▂▂▄▁
eval/steps_per_second,▇▁▂▁▆█▂▃▆██▇██▇▂▂▄▁
train/epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇██████
train/global_step,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇██████
train/learning_rate,██▇▇▆▆▆▅▅▄▄▃▃▃▂▂▁▁


In [15]:
print(best_results)

{'CASE_NUMBER': {'precision': 0.5691489361702128, 'recall': 0.7086092715231788, 'f1': 0.6312684365781711, 'number': 151}, 'COURT': {'precision': 0.7718120805369127, 'recall': 0.8424908424908425, 'f1': 0.8056042031523644, 'number': 273}, 'DATE': {'precision': 0.5501730103806228, 'recall': 0.9137931034482759, 'f1': 0.6868250539956804, 'number': 348}, 'GPE': {'precision': 0.7149321266968326, 'recall': 0.626984126984127, 'f1': 0.6680761099365751, 'number': 252}, 'JUDGE': {'precision': 0.8278688524590164, 'recall': 0.9017857142857143, 'f1': 0.8632478632478632, 'number': 112}, 'LAWYER': {'precision': 0.03389830508474576, 'recall': 0.6666666666666666, 'f1': 0.06451612903225806, 'number': 12}, 'ORG': {'precision': 0.5417789757412399, 'recall': 0.6633663366336634, 'f1': 0.5964391691394659, 'number': 303}, 'OTHERS': {'precision': 0.7571405518799419, 'recall': 0.8443404714774159, 'f1': 0.7983665135273098, 'number': 5557}, 'OTHER_PERSON': {'precision': 0.3392857142857143, 'recall': 0.7648305084745

# Evaluating Test Dataset

In [17]:
wandb.init(name = "legalbart-gold-test")

In [18]:
test_dataset_tokenized = test_dataset.map(tokenize_all_labels, batched=True)
test_results = trainer.evaluate(test_dataset_tokenized)

Map:   0%|          | 0/867 [00:00<?, ? examples/s]

In [19]:
print(test_results)

{'eval_loss': 0.19476103782653809, 'eval_precision': 0.6957157784743991, 'eval_recall': 0.8318340829585208, 'eval_f1': 0.757710253783999, 'eval_accuracy': 0.9494899452788975, 'eval_runtime': 14.0847, 'eval_samples_per_second': 61.556, 'eval_steps_per_second': 12.354, 'epoch': 5.0}


In [20]:
wandb.finish()

eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/precision,▁
eval/recall,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/accuracy,0.94949
